In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from scipy.stats import wilcoxon
from matplotlib.lines import Line2D

In [41]:
df = pd.read_csv('simulation_results_with_distances_2024-09-13.csv', header=0)

In [42]:
df

,simulation,trial,start_time,end_time,duration,duration_no_distance,sequence,type,duration_seconds,diff
0,1,0,14:37:35,14:37:47,00:00:12,12,"bowl, milk, cup, spoon, breakfast-cereal",opm,31.57,19.57
1,1,1,14:37:50,14:38:01,00:00:11,11,"bowl, milk, cup, spoon, breakfast-cereal",opm,30.57,19.57
2,1,2,14:38:04,14:38:16,00:00:12,12,"bowl, milk, cup, spoon, breakfast-cereal",opm,31.57,19.57
3,1,3,14:38:19,14:38:31,00:00:12,12,"bowl, milk, cup, spoon, breakfast-cereal",opm,31.57,19.57
4,1,4,14:38:33,14:38:46,00:00:13,13,"bowl, milk, cup, spoon, breakfast-cereal",opm,32.57,19.57
...,...,...,...,...,...,...,...,...,...,...
95,5,5,15:26:20,15:26:58,00:00:38,38,"breakfast-cereal, cup, bowl, spoon, milk",baseline,56.02,18.02
96,5,6,15:27:01,15:27:37,00:00:36,36,"breakfast-cereal, cup, bowl, spoon, milk",baseline,54.02,18.02
97,5,7,15:27:41,15:28:20,00:00:39,39,"breakfast-cereal, cup, bowl, spoon, milk",baseline,57.02,18.02
98,5,8,15:28:24,15:29:01,00:00:37,37,"breakfast-cereal, cup, bowl, spoon, milk",baseline,55.02,18.02


In [43]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]

In [44]:
df[['duration_seconds', 'type']].groupby(['type']).mean()

,duration_seconds
type,
baseline,48.37
opm,46.63


In [45]:
duration_opm = df.loc[df['type'] == 'opm']['duration_seconds'].mean()

In [46]:
duration_baseline = df.loc[df['type'] == 'baseline']['duration_seconds'].mean()

In [47]:
durations = pd.Series([duration_baseline, duration_opm])

In [48]:
durations.pct_change() * 100

0         NaN
1   -3.597271
dtype: float64

In [49]:
opm = df['duration_seconds'].loc[df['type'] == 'opm'].tolist()
baseline = df['duration_seconds'].loc[df['type'] == 'baseline'].tolist()
len(opm), len(baseline)

(50, 50)

### stat analysis

In [50]:
stat, p = wilcoxon(opm, baseline)
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 241.000, p = 0.00007


### plot results

In [51]:
results = pd.DataFrame()

In [52]:
custom_cmap2 = ["#46327e", "#277f8e", "#4ac16d"]
custom_lines2 = [Line2D([0], [0], color=custom_cmap2[1], lw=8, alpha=0.5),
                Line2D([0], [0], color=custom_cmap2[2], lw=8, alpha=0.5)]

In [53]:
results['opm'] = opm
results['baseline'] = baseline
labels = [col for col in results.columns]

In [54]:
%matplotlib qt

ax = sns.violinplot(data=results,
                    palette='viridis',
                    cut=0,
                    inner='quartiles')

plt.setp(ax.collections, alpha=0.7)

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=24)

plt.ylabel('duration in seconds', fontsize=24)
#plt.tight_layout(pad=0.9)
plt.show()

/tmp/ipykernel_58526/3156388523.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=24)


In [55]:
simulations_opm = df['simulation'].loc[df['type'] == 'opm'].tolist()
simulations_baseline = df['simulation'].loc[df['type'] == 'baseline'].tolist()

In [56]:
simulations_opm == simulations_baseline

True

In [57]:
results['simulation'] = simulations_baseline

In [58]:
results.groupby('simulation').mean()

,opm,baseline
simulation,,
1,34.87,40.17
2,42.69,42.77
3,48.37,50.29
4,51.81,53.10
5,55.41,55.52


In [59]:
labels = [col for col in results.columns]

%matplotlib qt

ax2 = sns.violinplot(data=results.groupby('simulation').mean(),
                    palette=custom_cmap2[1:],
                    cut=1,
                    inner='quartiles',
                    saturation=.8)

plt.setp(ax2.collections, alpha=0.5)

#ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=20)
#ax2.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=20)
ax2.tick_params(axis='both', which='major', labelsize=20)

plt.ylabel('duration in seconds', fontsize=24)
plt.xlabel('simulation type', fontsize=24)

ax2.legend(custom_lines2, ['OPM', 'baseline'], loc='upper left', fontsize=24)

plt.show()

In [61]:
%matplotlib qt

ax2 = plt.figure(figsize=(12,8))

ax2 = sns.violinplot(x='simulation',
                     y='duration_seconds',
                     hue='type',
                     data=df,
                     palette=custom_cmap2[1:],
                     cut=0,
                     inner='quartiles',
                     saturation=.8)

#ax2.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize=18)

plt.ylabel('duration in seconds', fontsize=26)
plt.xlabel('simulation', fontsize=24)

plt.setp(ax2.collections, alpha=0.5)
ax2.legend(custom_lines2, ['OPM', 'baseline'], loc='upper left', fontsize=24)
ax2.tick_params(axis='both', which='major', labelsize=20)

#ax2 = plt.gca()
#leg = ax2.get_legend()
#leg.legendHandles[0].set_color(custom_cmap2[1])
#leg.legendHandles[1].set_color(custom_cmap2[2])

#legend = plt.legend()
#for lh in legend.legendHandles:
#    lh.set_alpha(0.5)

plt.tight_layout()
plt.margins(0.01)
plt.show()